In [ ]:
# Standard libraries
import os

import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
from tqdm import tqdm
from tqdm import notebook as tqdm_notebook
import seaborn as sns
from IPython.display import Markdown, display
import json
from datetime import datetime
import random

# For downloading pre-trained models
import urllib.request
from urllib.error import HTTPError

# PyTorch Lightning
import pytorch_lightning as pl

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# PyTorch geometric
import torch_geometric
import torch_geometric.data as geom_data
import torch_geometric.nn as geom_nn

# PL callbacks
from pytorch_lightning.callbacks import ModelCheckpoint
from torch import Tensor

import src.cropnet as cropnet

2025-04-10 11:52:49.062887: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 11:52:51.087294: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-10 11:52:53.305481: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<module 'cropnet' from '/home/oskar/Documents/Jupyter/Sentinel/cropnet.py'>

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [3]:
def gen_option(p_val=0.15, p_test=0.15, layer_sizes=lambda M: [M, M//2, 64, 1], seed=0, dropout=0.0, epochs=200, lr_schedule=None, pred_var=False):
    return {
        'p_val': p_val,
        'p_test': p_test,
        'layer_sizes': layer_sizes,
        'seed': seed,
        'dropout': dropout,
        'epochs': epochs,
        'lr_schedule': lr_schedule,
        'pred_var': pred_var,
    }

In [15]:
def get_dataset(path, config):
    df, correlation_filters = cropnet.load_data(path, filter_yield=config['filter_yield'], filter_wheat=config['filter_wheat'], filter_years=config['filter_years'], df=config['df'], correlation_filters=config['correlation_filters'])


    if config['target'] is not None:
        target = config['target']
    else:
        target = 'yield_1'

    
    if config['predictors'] is not None:
        predictors = config['predictors']
    else:
        predictors = [c for c in df.columns if c not in ['index', 'x', 'y', 'x_int', 'y_int', 'polygon_id', 'year', 'grdkod_mar'] and "yield" not in c and "_pid" not in c]

    if config['filter_weeks'] is not None:
        predictors = [p for p in predictors if (not p.startswith("week")) or (int(p.split("_")[1]) == config['filter_weeks'])]

    norm_info = cropnet.get_norm_info(df, target, predictors)

    if config['type'] == 'futurum':
        dataset = cropnet.create_futurum_dataset(df, target, predictors, config['target_year'], config['pred_years'], norm_info, device, dist_thresh=config['dist_thresh'], sparsity_thresh=config['sparsity_thresh'])
    else:
        if config['time_scheme'] == 'concat':
            dataset = []
            for i in range(2017+1, 2022):
                dataset += cropnet.create_futurum_dataset_2(df, target, predictors, i, [i-1,i], norm_info, device, dist_thresh=config['dist_thresh'], sparsity_thresh=config['sparsity_thresh'])
        else:
            dataset, skipped = cropnet.create_present_dataset(df, target, predictors, norm_info, device, dist_thresh=config['dist_thresh'], time_scheme=config['time_scheme'], size_thresh=config['size_thresh'], sparsity_thresh=config['sparsity_thresh'])
        
    M = dataset[1][0].x.shape[1]
    print("Number of features:", M)
    print("Number of fields:", len(dataset))
    print("Number of points (field 0):", len(dataset[0][0].x))
    print("Number of connections (field 0):", len(dataset[0][0].edge_index[0]))
    
    return dataset, norm_info

In [31]:

def run_model(p_val, p_test, dataset, norm_info, device, layer_sizes, seed=None, pred_var=False, lr_schedule=None, dropout=0.0, epochs=200, loss_function=cropnet.Loss_MSE):
    
    if seed is not None: 
        print("Setting seed:", seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.manual_seed(seed)
    
    if lr_schedule is not None:
        lr_schedule = sorted(lr_schedule)
    
    print(layer_sizes)
    model = cropnet.GCN_v2(layer_sizes, dropout=dropout, edge_dim=dataset[0][0].edge_features.shape[1])
    model.to(device)

    history = {
        'train': [],
        'val': [],
        'test': []
    }
    

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = loss_function

    scores = {}
    top_score = 10000
    
    data_train, data_val, data_test = cropnet.split_data(dataset, p_val, p_test, seed=seed)

    # Training loop
    model.train()

    pbar = tqdm(range(epochs))
    for epoch in pbar:
        
        random.shuffle(dataset)
        err = np.zeros(0)
        total_los = []
        for X, y, info in dataset:
            optimizer.zero_grad()
            pred = model(X)
            y_cpu = y.clone().cpu().detach().numpy()
            pred_cpu = pred.clone().cpu().detach().numpy()
            if pred_var:
                y_cpu = y_cpu - np.mean(y_cpu) + np.mean(pred_cpu)
            y_cpu[np.isnan(y_cpu)] = pred_cpu[np.isnan(y_cpu)]
            #if 'valid' in info: y_cpu[info['valid'] == False] = pred_cpu[info['valid'] == False]
            y_gpu = torch.tensor(y_cpu).to(device)
            loss = criterion(pred, y_gpu)
            loss.backward()
            total_los.append(loss.item())
            optimizer.step()
            err = np.concatenate([err, cropnet.eval_error(pred, y, info, norm_info)])
        train_rmse = np.sqrt(np.nanmean(err**2))
        history['train'].append(train_rmse)

        optimizer.zero_grad()

        val_rmse, val_err = cropnet.eval_dataset(data_val, model, norm_info)
        #val_err = val_err - np.nanmean(val_err)
        history['val'].append(val_rmse)
        val_acc10 = np.mean((np.abs(val_err) < 1)[np.isnan(val_err) == False])
        metric = val_rmse
            
        test_rmse, test_err = cropnet.eval_dataset(data_test, model, norm_info)
        #test_err = test_err - np.nanmean(test_err)
        #test_rmse = np.sqrt(np.nanmean(test_err**2))
        history['test'].append(test_rmse)
        acc5 = np.mean((np.abs(test_err) < 0.5)[np.isnan(test_err) == False])
        acc10 = np.mean((np.abs(test_err) < 1)[np.isnan(test_err) == False])
        acc20 = np.mean((np.abs(test_err) < 2)[np.isnan(test_err) == False])

        if lr_schedule is not None: 
            cropnet.update_lr(optimizer, lr_schedule, val_rmse)

        if metric < top_score:
            top_score = metric
            torch.save(model, "/".join(["models", "current.pth"]))
            scores = {
                'test_rmse': test_rmse,
                'test_acc_0.5': acc5,
                'test_acc_1.0': acc10, 
                'test_acc_2.0': acc20,
                'test_rel_rmse': test_rmse/norm_info['y']['mean'],
            }

        pbar.set_description(f'Loss: {np.round(np.mean(total_los), 3)}, train rmse: {np.round(train_rmse, 3)}, val rmse: {np.round(val_rmse, 3)}, test rmse: {np.round(test_rmse, 3)}, top: [{np.round(metric, 3)} >= {np.round(top_score, 3)} {[np.round(scores[s], 3) for s in scores]}]')

    model = torch.load("models/current.pth")
    return model, history, scores, (data_train, data_val, data_test)

In [32]:
def test_run(config, dataset=None, norm_info=None):
    
    if dataset is None:
        path = "data/full_data/processed/processed_merged.feather"
        dataset, norm_info = get_dataset(path, config)
        
    M = dataset[1][0].x.shape[1]

    score_list, history_list, data_list, pred_list = [], [], [], []
    for option in config['options']:
        model, history, scores, data = run_model(
            p_val=option['p_val'], 
            p_test=option['p_test'], 
            layer_sizes=option['layer_sizes'](M),
            seed=option['seed'],  
            dropout=option['dropout'],  
            epochs=option['epochs'], 
            lr_schedule=option['lr_schedule'], 
            pred_var=option['pred_var'],  
            dataset=dataset, 
            norm_info=norm_info, 
            device=device,  
        )
    
        data_train, data_val, data_test = data
        
        y_tot = np.zeros(0)
        y_pred = np.zeros(0)
        test_err = np.zeros(0)
        test_err_norm = np.zeros(0)
        pids = np.zeros(0)
        for X, y, info in data_test:
            pred = model(X)
            
            yp = pred.cpu().detach().numpy().reshape(-1)
            yt = y.cpu().detach().numpy().reshape(-1)
            err = (yt - yp)*norm_info['y']['std']
            
            y_tot = np.concatenate([y_tot, yt])
            y_pred = np.concatenate([y_pred, yp])
            test_err = np.concatenate([test_err, err])
            test_err_norm = np.concatenate([test_err_norm, err - np.nanmean(err)])
            pids = np.concatenate([pids, np.ones(err.shape)*info['pid']])
            
        y_tot = y_tot*norm_info['y']['std'] + norm_info['y']['mean']
        scores_unnorm = [np.sqrt(np.nanmean(test_err**2)), np.nanmean((np.abs(test_err) < 1)[np.isnan(test_err) == False]), np.nanmean(np.abs(test_err/y_tot)), 1-np.sum(test_err**2)/np.sum((y_tot-np.mean(y_tot))**2)]
        scores_norm = [np.sqrt(np.nanmean(test_err_norm**2)), np.nanmean((np.abs(test_err_norm) < 1)[np.isnan(test_err_norm) == False]), np.nanmean(np.abs(test_err_norm/y_tot)), 1-np.sum(test_err_norm**2)/np.sum((y_tot-np.mean(y_tot))**2)]
        result = [scores_unnorm, scores_norm, [list(y_tot), list(y_pred), list(test_err), list(test_err_norm), list(pids)]]
        
        #history_list.append(history)
        score_list.append(result)
        torch.cuda.empty_cache()
        # preds = [(
        #     model(X).cpu().detach().numpy().reshape(-1), 
        #     y.cpu().detach().numpy().reshape(-1), 
        #     info
        # ) for X, y, info in data[2]]
        #pred_list.append(preds)
        print([scores_unnorm, scores_norm])
        
    #data_list.append(data)
    return score_list, model, data#, history_list, data_list, pred_list, norm_info


In [33]:

path = "data/full_data/processed/processed_merged.feather"
df = pd.read_feather(path)
f = open(path.replace(".feather", ".json"), "r")
correlation_filters = json.load(f)
f.close()

replaces = []
weeks = set(["_".join(d.split("_")[:2]) + "_" for d in df.columns if "week_" in d])
for week in weeks:
    replaces += [(d, d.replace(week, week + "weather_")) for d in df.columns if d.startswith(week)]

dates = [d.split("_")[0] for d in df.columns if "sigma0_vh_norm_multi" in d]
weeks = ["week_" + str(week) for week in range(16, 16+len(dates))]
for i in range(len(weeks)):
    replaces += [(d, d.replace(dates[i], weeks[i])) for d in df.columns if d.startswith(dates[i])]

replaces = {r[0]: r[1] for r in replaces}

df = df.rename(replaces, axis=1)

In [34]:

target = 'yield_1'
predictors = [c for c in df.columns if c not in ['index', 'x', 'y', 'x_int', 'y_int', 'polygon_id', 'year', 'grdkod_mar'] and "yield" not in c and "_pid" not in c]

norm_info = cropnet.get_norm_info(df, target, predictors)

In [35]:
if False:
    sparsities = []
    for pid in set(df['matching_pid']):
        pdf = df[df['matching_pid'] == pid]
        for year in set(pdf['year']):
            ydf = pdf[pdf['year'] == year]
            count = np.sum(np.isnan(ydf['yield']) == False)
            if count > 0:
                sparsities.append((count/len(ydf), pid, year))
                
    for spar in sparsities:
        if spar[0] < 0.4:
            df.loc[(df['matching_pid'] == spar[1]) & (df['year'] == spar[2]), 'yield'] = np.nan
        
        

# Test Run

In [36]:
config_futurum = {
    'type': "futurum",
    'filter_yield': False,
    'filter_wheat': False,
    'filter_years': False,
    'filter_weeks': None,
    'predictors': None,
    'target': 'yield_1',
    'dist_thresh': 200,
    'sparsity_thresh': None,
    'target_year': 2019,
    'pred_years': [2017, 2018],
    'options': [
        gen_option(layer_sizes=lambda M: [M, M//2, M//4, M//8, 64, 1], epochs=100),
    ],
    'df': df,
    'correlation_filters': correlation_filters,
}

config_present = {
    'type': "present",
    'filter_yield': True,
    'filter_wheat': True,
    'filter_years': False,
    'filter_weeks': None,
    'predictors': None,
    'target': 'yield_1',
    'dist_thresh': 200, 
    'sparsity_thresh': None,
    'size_thresh': None,
    'time_scheme': None,
    'options': [gen_option(epochs=100, seed=seed) for seed in range(1)],
    'df': df,
    'correlation_filters': correlation_filters,
}

In [37]:
scores, model, data = test_run(config_present)


2198937 (954): 100%|██████████| 152/152 [00:23<00:00,  6.50it/s]


Number of features: 891
Number of fields: 152
Number of points (field 0): 1815
Number of connections (field 0): 15375
Setting seed: 0
[891, 445, 64, 1]


Loss: 0.098, train rmse: 0.802, val rmse: 0.672, test rmse: 0.645, top: [0.672 >= 0.669 [0.669, 0.621, 0.884, 0.988, 0.081]]: 100%|██████████| 100/100 [01:30<00:00,  1.10it/s]

[[0.668917243406162, 0.8840729922422187, 0.07172592334001338, 0.8817361850994914], [0.6505499503212981, 0.89531706397814, 0.06962570766342768, 0.888141652926046]]


In [ ]:
torch.save(model, "models/final_example/final_example.pt")

In [ ]:
def get_dataset(path, config):
    df, correlation_filters = cropnet.load_data(path, filter_yield=config['filter_yield'], filter_wheat=config['filter_wheat'], filter_years=config['filter_years'], df=config['df'], correlation_filters=config['correlation_filters'])


    if config['target'] is not None:
        target = config['target']
    else:
        target = 'yield_1'

    
    if config['predictors'] is not None:
        predictors = config['predictors']
    else:
        predictors = [c for c in df.columns if c not in ['index', 'x', 'y', 'x_int', 'y_int', 'polygon_id', 'year', 'grdkod_mar'] and "yield" not in c and "_pid" not in c]

    if config['filter_weeks'] is not None:
        predictors = [p for p in predictors if (not p.startswith("week")) or (int(p.split("_")[1]) == config['filter_weeks'])]

    norm_info = cropnet.get_norm_info(df, target, predictors)

    if config['type'] == 'futurum':
        dataset = cropnet.create_futurum_dataset(df, target, predictors, config['target_year'], config['pred_years'], norm_info, device, dist_thresh=config['dist_thresh'], sparsity_thresh=config['sparsity_thresh'])
    else:
        if config['time_scheme'] == 'concat':
            dataset = []
            for i in range(2017+1, 2022):
                dataset += cropnet.create_futurum_dataset(df, target, predictors, i, [i-1,i], norm_info, device, dist_thresh=config['dist_thresh'], sparsity_thresh=config['sparsity_thresh'])
        else:
            dataset, skipped = cropnet.create_present_dataset(df, target, predictors, norm_info, device, dist_thresh=config['dist_thresh'], time_scheme=config['time_scheme'], size_thresh=config['size_thresh'], sparsity_thresh=config['sparsity_thresh'])
        
    M = dataset[1][0].x.shape[1]
    print("Number of features:", M)
    print("Number of fields:", len(dataset))
    print("Number of points (field 0):", len(dataset[0][0].x))
    print("Number of connections (field 0):", len(dataset[0][0].edge_index[0]))
    
    return dataset, norm_info

In [51]:
config = {
    'type': "present",
    'filter_yield': True,
    'filter_wheat': True,
    'filter_years': False,
    'filter_weeks': None,
    'predictors': None,
    'target': 'yield_1',
    'dist_thresh': 200, 
    'sparsity_thresh': None,
    'size_thresh': None,
    'time_scheme': None,
    'options': [gen_option(epochs=100, seed=seed) for seed in range(1)],
    'df': df,
    'correlation_filters': correlation_filters,
}
path = "data/full_data/processed/processed_merged.feather"
dataset, norm_info = get_dataset(path, config)

2198937 (954): 100%|██████████| 152/152 [00:25<00:00,  5.94it/s]

Number of features: 891
Number of fields: 152
Number of points (field 0): 1815
Number of connections (field 0): 15375


In [ ]:
model = torch.load("models/final_example/final_example.pt", weights_only=False)

X, y, info = dataset[0]
p = model(X)
err = (y-p).cpu().detach().numpy()
print(np.sqrt(np.nanmean(err**2))*norm_info['y']['std'])

0.5094963057137166


In [10]:

scores, model, data = test_run(config_present)
del model, data
cropnet.save_score_to_json(scores, "results_final/baseline/score.json")

KeyboardInterrupt: 

In [ ]:
# Scheme Ablation
#[None, "distance", "one-hot", "concat"]
for i, scheme in enumerate([None, "distance", "one-hot", "concat"]):
    config_present['time_scheme'] = scheme
    config_present['filter_yield'] = scheme != "concat"
    config_present['filter_wheat'] = scheme != "concat"
    config_present['dist_thresh'] = 200 if scheme != "concat" else 100
    scores, model, data = test_run(config_present)
    del model, data
    cropnet.save_score_to_json(scores, "results_final/present_scheme/score" + str(i) + ".json")

2198937 (954): 100%|██████████| 152/152 [00:25<00:00,  6.05it/s]


Number of features: 891
Number of fields: 152
Number of points (field 0): 1815
Number of connections (field 0): 15375
Setting seed: 0
[891, 445, 64, 1]


Loss: 0.081, train rmse: 0.694, val rmse: 1.116, test rmse: 0.936, top: [1.116 >= 0.954 [0.889, 0.475, 0.768, 0.968, 0.108]]: 100%|██████████| 100/100 [01:04<00:00,  1.55it/s]


[[0.8891391571773283, 0.7684443606897201, 0.09596687318878531, 0.7910479795431], [0.8233902112736254, 0.8209742768303024, 0.08698678702917703, 0.8208080478693346]]
Setting seed: 1
[891, 445, 64, 1]


Loss: 0.083, train rmse: 0.719, val rmse: 0.943, test rmse: 0.86, top: [0.943 >= 0.938 [0.847, 0.485, 0.791, 0.977, 0.103]]: 100%|██████████| 100/100 [01:04<00:00,  1.56it/s]


...

[[0.7624130517964861, 0.8374299266925399, 0.0931354008695549, 0.8767107619608849], [0.6332956499249645, 0.9069796094375655, 0.07749653456955229, 0.9149337019328553]]
Setting seed: 8
[1782, 891, 64, 1]


Loss: 0.09, train rmse: 0.98, val rmse: 1.071, test rmse: 1.199, top: [1.071 >= 0.965 [0.964, 0.436, 0.746, 0.952, 0.125]]: 100%|██████████| 100/100 [01:26<00:00,  1.16it/s] 


[[0.9638428937680956, 0.7456407137064072, 0.09496338567214072, 0.7843689505090712], [0.7817521629508435, 0.8527777777777777, 0.07608243987871353, 0.8581475045656859]]
Setting seed: 9
[1782, 891, 64, 1]


Loss: 0.067, train rmse: 0.768, val rmse: 1.184, test rmse: 1.395, top: [1.184 >= 0.943 [0.983, 0.465, 0.756, 0.947, 0.127]]: 100%|██████████| 100/100 [01:27<00:00,  1.15it/s]


[[0.9829876131525709, 0.7562690943388489, 0.1053725231300547, 0.7930882845361369], [0.7253173866082581, 0.8879046974636566, 0.07768825829501647, 0.8873463760535294]]


In [ ]:
config_present = {
    'type': "present",
    'filter_yield': True,
    'filter_wheat': True,
    'filter_years': False,
    'filter_weeks': None,
    'predictors': None,
    'target': 'yield_1',
    'dist_thresh': 200, 
    'sparsity_thresh': None,
    'size_thresh': None,
    'time_scheme': None,
    'options': [gen_option(epochs=100, seed=seed) for seed in range(10)],
    'df': df,
    'correlation_filters': correlation_filters,
}

for i, week in enumerate([16, 18, 20, 22, 24, 26, 28]):
    config_present['filter_weeks'] = week
    scores, model, data = test_run(config_present)
    cropnet.save_score_to_json(scores, "results_final/present_week/score" + str(i) + ".json")

2198937 (954): 100%|██████████| 152/152 [00:21<00:00,  6.91it/s]


Number of features: 67
Number of fields: 152
Number of points (field 0): 1815
Number of connections (field 0): 15375
Setting seed: 0
[67, 33, 64, 1]


Loss: 0.138, train rmse: 0.925, val rmse: 1.505, test rmse: 1.343, top: [1.505 >= 1.391 [1.22, 0.35, 0.623, 0.903, 0.148]]:  80%|████████  | 80/100 [00:28<00:07,  2.84it/s] 

Loss: 0.13, train rmse: 0.89, val rmse: 1.416, test rmse: 1.331, top: [1.416 >= 1.391 [1.22, 0.35, 0.623, 0.903, 0.148]]: 100%|██████████| 100/100 [00:35<00:00,  2.82it/s] 


[[1.2197866218925224, 0.6231037406953736, 0.1439027987273635, 0.6067444747511654], [1.0433857970229128, 0.7307547347592576, 0.1191566598328618, 0.7122621841275734]]
Setting seed: 1
[67, 33, 64, 1]


...

[[1.124242789753093, 0.7126579422382672, 0.11784817252443643, 0.7650638621456618], [0.9523096943073895, 0.7639891696750902, 0.1022861451690779, 0.8314277770209388]]
Setting seed: 8
[65, 32, 64, 1]


Loss: 0.086, train rmse: 0.699, val rmse: 0.914, test rmse: 1.16, top: [0.914 >= 0.889 [1.089, 0.369, 0.649, 0.944, 0.132]]: 100%|██████████| 100/100 [00:36<00:00,  2.72it/s]


[[1.0894465942093656, 0.6487381479038121, 0.13153415934715998, 0.7831802657924792], [0.982724964095438, 0.7075814609285058, 0.11583293377781469, 0.8235787504723386]]
Setting seed: 9
[65, 32, 64, 1]


Loss: 0.093, train rmse: 0.722, val rmse: 1.058, test rmse: 0.918, top: [1.058 >= 0.982 [0.991, 0.477, 0.749, 0.953, 0.12]]: 100%|██████████| 100/100 [00:35<00:00,  2.84it/s]


[[0.9905412861904733, 0.7489198743126473, 0.10589146005062255, 0.7634728220475304], [0.8070457363279332, 0.8305184603299293, 0.08768844469056732, 0.8429882401356126]]


In [12]:
rem = [c for c in df.columns if c not in ['index', 'x', 'y', 'x_int', 'y_int', 'polygon_id', 'year', 'grdkod_mar'] and "yield" not in c and "_pid" not in c]

soil = [d for d in rem if 'week' not in d]
rem = [d for d in rem if d not in soil]
weather = [d for d in rem if 'weather' in d]
rem = [d for d in rem if d not in weather]
sen1 = [d for d in rem if 'sigma' in d or 'theta' in d]
sen2 = [d for d in rem if d not in sen1]

In [13]:
config_present = {
    'type': "present",
    'filter_yield': True,
    'filter_wheat': True,
    'filter_years': False,
    'filter_weeks': None,
    'predictors': None,
    'target': 'yield_1',
    'dist_thresh': 200, 
    'sparsity_thresh': None,
    'size_thresh': None,
    'time_scheme': None,
    'options': [gen_option(epochs=100, seed=seed) for seed in range(10)],
    'df': df,
    'correlation_filters': correlation_filters,
}

runs = [sen1+sen2+weather+soil, sen2+weather+soil, sen1+weather+soil, sen1+sen2+soil, sen1+sen2+weather, sen1+sen2, sen1+weather, sen1+soil, sen2+weather, sen2+soil, weather+soil, sen1, sen2, weather, soil]
for i, predictors in enumerate(runs):
    config_present['predictors'] = predictors
    scores, model, data = test_run(config_present)
    cropnet.save_score_to_json(scores, "results_final/present_feature/score" + str(i) + ".json")

2198937 (954): 100%|██████████| 152/152 [00:23<00:00,  6.42it/s]


Number of features: 891
Number of fields: 152
Number of points (field 0): 1815
Number of connections (field 0): 15375
Setting seed: 0
[891, 445, 64, 1]


Loss: 0.118, train rmse: 0.811, val rmse: 1.123, test rmse: 0.906, top: [1.123 >= 1.08 [0.997, 0.36, 0.699, 0.957, 0.121]]:  37%|███▋      | 37/100 [00:23<00:40,  1.56it/s]  

Loss: 0.095, train rmse: 0.747, val rmse: 1.124, test rmse: 0.921, top: [1.124 >= 0.953 [0.944, 0.408, 0.73, 0.964, 0.114]]: 100%|██████████| 100/100 [01:04<00:00,  1.55it/s]


[[0.9437131376314337, 0.7299381261974308, 0.1030231385034988, 0.7646104907857056], [0.8335743219242776, 0.8079556518734885, 0.08959545126406497, 0.8163479599143382]]
Setting seed: 1
[891, 445, 64, 1]


...

[[2.0388128836990544, 0.40083483754512633, 0.318812556122169, 0.22734727282460976], [1.4232614872169742, 0.5711223568849922, 0.1701533600057829, 0.6234705077884548]]
Setting seed: 8
[22, 11, 64, 1]


Loss: 0.453, train rmse: 1.77, val rmse: 2.192, test rmse: 2.641, top: [2.192 >= 2.025 [2.585, 0.138, 0.267, 0.497, 0.313]]: 100%|██████████| 100/100 [00:36<00:00,  2.73it/s]


[[2.585408965364059, 0.26698746555939307, 0.35420696070846275, -0.2210836538224843], [2.101137345088953, 0.37212671556262694, 0.2639744546604938, 0.1935159171798948]]
Setting seed: 9
[22, 11, 64, 1]


Loss: 0.482, train rmse: 1.963, val rmse: 2.195, test rmse: 2.328, top: [2.195 >= 1.911 [1.788, 0.267, 0.492, 0.766, 0.217]]: 100%|██████████| 100/100 [00:34<00:00,  2.86it/s]


[[1.788357382710023, 0.4924554857292485, 0.22293638316857411, 0.2290175179249201], [1.3529472795284363, 0.6332645980623199, 0.15276951137588593, 0.5587368774058523]]
